# CS 3110/5110: Data Privacy
## Homework 1

In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np

adult = pd.read_csv('https://github.com/jnear/cs3110-data-privacy/raw/main/homework/adult_with_pii.csv')
adult_pii = adult[['Name', 'DOB', 'SSN', 'Zip', 'Age']]
adult_deid = adult.drop(columns=['Name', 'SSN'])

## Question 1 (20 points)

Using the dataframes `adult_pii` and `adult_deid`, write code to conduct a linking attack to recover the names of as many individuals in `adult_deid` as possible. Your solution should be parameterized by the set of columns to use in the attack.

In [2]:
def linking_attack(cols):
    return pd.merge(adult_pii, adult_deid, left_on=cols, right_on=cols)

In [3]:
# TEST CASES for Question 1

assert len(linking_attack(['Zip'])) == 43191
assert len(linking_attack(['Zip', 'DOB'])) == 32563
assert len(linking_attack(['Zip', 'Age'])) == 32755

## Question 2 (5 points)

How many individuals in this dataset are uniquely identified by their Zip code? How many are uniquely identified by their age?

Hint: note that the number of *unique ZIP codes* is **different** from the number of *individuals uniquely identified by ZIP code*.

Hint: you can use the `value_counts` method to count the number of occurences of each value in a series.

In [4]:
def unique_zipcode():
    attack = linking_attack(['Zip'])
    freq = attack['Name'].value_counts()
    count = 0
    for name in freq:
        if name == 1:
            count += 1
    return count

def unique_dob():
    attack = linking_attack(['DOB'])
    freq = attack['Name'].value_counts()
    count = 0
    for name in freq:
        if name == 1:
            count += 1
    return count

print(unique_zipcode())
print(unique_dob())

# There has to be a better way to get this count

23513
7845


In [5]:
# TEST CASES (INCOMPLETE) for Question 2
# INCOMPLETE means that additional test cases will be used to check your answer during grading

assert unique_zipcode() > 0
assert unique_dob() > 0


## Question 3 (10 points)

Write code to determine the `Education-Num` of the individual named Ardyce Golby by performing a differencing attack. Your code should *only* use aggregate data to find Ardyce's education number.

In [6]:
def ardyce_education():
    attack = linking_attack(['DOB', 'Zip'])
    return attack['Education-Num'].sum() - attack[attack['Name'] != 'Ardyce Golby']['Education-Num'].sum()

In [7]:
# TEST CASE for Question 3
assert ardyce_education() == 12